In [ ]:
import cv2
from gui_buttons import Buttons

In [ ]:
import numpy as np

#initialize buttons
button = Buttons()
button.add_button("person",20,20)
button.add_button("cell phone", 20, 100)
button.add_button("keyboard", 20, 180)
button.add_button("remote", 20, 260)
button.add_button("car", 20, 340)
colors = button.colors
#open cv dnn



net = cv2.dnn.readNet("C:\\Users\\tarun\\Desktop\\customdetection\\dnn_model\\yolov4-tiny.weights","C:\\Users\\tarun\\Desktop\\customdetection\\dnn_model\\yolov4-tiny.cfg")
model = cv2.dnn_DetectionModel(net)
model.setInputParams(size = (320,320),scale = 1/255)



#loading classes list

classes = []
with open("C:\\Users\\tarun\\Desktop\\customdetection\\dnn_model\\classes.txt","r") as file_object:
    for class_name in file_object.readlines():
        class_name = class_name.strip()
        classes.append(class_name)
print(classes)


##############################################################
#using camera parameters to increase resolution
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)

#####################################


def click_button(event,x,y,flags,params):
    global button_person
    if event == cv2.EVENT_LBUTTONDOWN:
        button.button_click(x,y)
    

#create window
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame",click_button)


while True:
    
    #get frames
    ret , frame = cap.read()
    
    #get active button list
    
    active_buttons = button.active_buttons_list()
    
    
    #object detection 
    (class_ids, scores , bboxes) = model.detect(frame)
    #zip is used  to collect individual elements from above parameters
    for class_id, score ,bbox in zip(class_ids, scores , bboxes):
        (x,y,w,h) = bbox
        class_name =classes[class_id]
        
        if class_name in active_buttons:
            #class id is a number which consists of custom names in classes.txt
            cv2.putText(frame,class_name,(x,y-5),cv2.FONT_HERSHEY_PLAIN,2,(200,0,0),2)
            cv2.rectangle(frame,(x,y),(x+w , y + h),(200,0,50),3)
   
    #DISPLAY BUTTON
    button.display_buttons(frame)
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#cx freeze is used to convert python file to executable file

import sys
from cx_Freeze import setup, Executable
setup(name="Simple Object Detection Software",
      version="0.1",
      description="This software detects objects in realtime",
      executables=[Executable("custom object detection yolov4.ipynb")]
      )